In [12]:
# ❶ 라이브러리 로드 & 원본 JSON 읽기
import json, pathlib, pprint

SRC = pathlib.Path("/Users/taeyoonkwack/Documents/HCLT-KACL-2025/Korean_Dialouge_Inference/dataset/dev_kor.json")
DST = pathlib.Path("/Users/taeyoonkwack/Documents/HCLT-KACL-2025/Korean_Dialouge_Inference/dataset/finetuning_data/dev_kor.json")

with SRC.open(encoding="utf-8") as f:
    raw = json.load(f)

print(f"{len(raw):,} samples loaded")

151 samples loaded


In [13]:
def conversation_to_str(conv):
    """
    [
      {"speaker": 2, "utterance": "안녕"},
      {"speaker": 1, "utterance": "반가워"}
    ] -> "화자 2: 안녕\n화자 1: 반가워"
    """
    return "\n".join(f"화자 {turn['speaker']}: {turn['utterance']}"
                     for turn in conv)

def key_to_letter(key):
    """'inference_1'|'_2'|'_3' -> 'A'|'B'|'C'"""
    return {"inference_1": "A", "inference_2": "B", "inference_3": "C"}[key]

In [14]:
processed = []
SYSTEM_PROMPT = "당신은 대화 맥락을 추론하는 assistant입니다. 아래의 대화 내용을 참고하여 문제의 정답을 맞추시오. "

for sample in raw:
    inp  = sample["input"]
    conv = conversation_to_str(inp["conversation"])

    user_msg = f"""[대화]
{conv}

[질문]
아래의 선지 중, {inp['question']}

[선지]
A. {inp['inference_1']}
B. {inp['inference_2']}
C. {inp['inference_3']}

A, B, C 를 제외한 그 무엇도 출력하지 마시오. 
Return your answer only — no additional text."""
    assistant_msg = key_to_letter(sample["output"])

    processed.append({
        "id": sample["id"],
        "messages": [
            {"role": "system",    "content": SYSTEM_PROMPT},
            {"role": "user",      "content": user_msg},
            {"role": "assistant", "content": assistant_msg}
        ]
    })
print("example =>"); pprint.pprint(processed[0], depth=3)

example =>
{'id': 'nikluge-2025-대화 맥락 추론-dev-000001',
 'messages': [{'content': '당신은 대화 맥락을 추론하는 assistant입니다. 아래의 대화 내용을 참고하여 문제의 '
                          '정답을 맞추시오. ',
               'role': 'system'},
              {'content': '[대화]\n'
                          '화자 1: 안녕하세요! ㅎㅎ\n'
                          '화자 2: 안녕하세요!\n'
                          '화자 2: 동호회활동하세요?\n'
                          '화자 1: 저는 차 마시는 거 좋아해서 차 좋아하는 사람들이랑 가끔 다회 참석해요\n'
                          '화자 1: name2님은 동호회활동 참여하는 거 있으신가요?\n'
                          '화자 2: 오오\n'
                          '화자 2: 아니요ㅋㅋㅋ저는 없어요\n'
                          '화자 2: 새로운 사람 만나는 걸 즐기지는 않아서 아직 시도를 못 해봤어요 동호회!\n'
                          '화자 1: 아하ㅎㅎ  저는 직업상 만나는 사람들이 한정적이어서 동호회 활동을 하니까 다양한 '
                          '사람을 만나게 되어 좋더라고요\n'
                          '화자 1: 저도 굉장히 내향적인 편인데 동호회 안에서 친해진 사람들하고만 만나고 있어요\n'
                          '화자 2: 이상하게 동호회는 목적이 다르다는 편견이 좀 있어서 망설여지더라고요!\n'
                          '화자 2: 물론 다

In [11]:
with DST.open("w", encoding="utf-8") as f:
    json.dump(processed, f, ensure_ascii=False, indent=2)

print(f"✔️ Saved {len(processed):,} samples → {DST.resolve()}")

✔️ Saved 151 samples → /Users/taeyoonkwack/Documents/HCLT-KACL-2025/Korean_Dialouge_Inference/dataset/finetuning_data/dev_kor.json
